In [1]:
import pandas as pd
import os,sys
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import urllib
import sqlite3
import json
import time
import ssl
import pandas as pd
import numpy as np
import re as re

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
## the files from data.sg come in three csv files, i read all three here
filestrain= sorted([a for a in os.listdir('.') if (a.endswith('.csv') and a.startswith('resale'))])
train=pd.read_csv(filestrain[0], header=0)
for i in filestrain[1:]:
    train = pd.concat([pd.read_csv(i, header=0),train])
train=train.sort_values(by='month').reset_index()

target='resale_price'
predictors=[x for x in  list(train.keys()) if x!=target]
train['sql_name']=train['street_name']+' '+train['block']+'  SINGAPORE'

In [3]:
train.head()

,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,sql_name
0,679,1990-01,GEYLANG,3 ROOM,40,CIRCUIT RD,01 TO 03,57.0,STANDARD,1971,23400.0,CIRCUIT RD 40 SINGAPORE
1,823,1990-01,JURONG EAST,5 ROOM,209,JURONG EAST ST 21,07 TO 09,123.0,IMPROVED,1983,109000.0,JURONG EAST ST 21 209 SINGAPORE
2,822,1990-01,JURONG EAST,5 ROOM,258,JURONG EAST ST 24,07 TO 09,121.0,IMPROVED,1983,135000.0,JURONG EAST ST 24 258 SINGAPORE
3,821,1990-01,JURONG EAST,5 ROOM,240,JURONG EAST ST 21,01 TO 03,121.0,IMPROVED,1982,109000.0,JURONG EAST ST 21 240 SINGAPORE
4,820,1990-01,JURONG EAST,5 ROOM,240,JURONG EAST ST 21,04 TO 06,121.0,IMPROVED,1982,109000.0,JURONG EAST ST 21 240 SINGAPORE


In [4]:
''' 
writing out the unique block names in the dataset, and getting 
GPS cordinates from google API later. 
'''
x=tuple(train.groupby(['street_name','block']))
x=sorted(x)
f1=open('0.out','w')
for i in range(len(x)):
    if i%2200 == 0: 
        f1.close()
        f1=open(str(i)+'.out','w')
    f1.write(x[i][0][0]+' '+x[i][0][1]+'  Singapore\n') 
f1.close()

In [5]:
### now get the lat,lng cordinates from google API
serviceurl = "http://maps.googleapis.com/maps/api/geocode/json?"

# Deal with SSL certificate anomalies Python > 2.7
#scontext = ssl.SSLContext(ssl.PROTOCOL_TLSv1)
#scontext = None

conn = sqlite3.connect('geodata_HDB_SG.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

#this is limited to 2500 each days, so there are four here , to be done in four days
fh = open("6600.out") 
#fh = open("4400.out")
#fh = open("2200.out")
#fh = open("0.out")
#fh = open('where.data_mrt') # file containing all the mrt stations
count = 0
for line in fh:
    if count > 25 : break #just showing 25 examples.
    address = line.strip()
    print ''
    cur.execute("SELECT geodata FROM Locations WHERE address= ?", (buffer(address), ))

    try:
        data = cur.fetchone()[0]
        print "Found in database ",address
        continue
    except:
        pass

    print 'Resolving', address
    url = serviceurl + urllib.urlencode({"sensor":"false", "address": address})
    print 'Retrieving', url
    uh = urllib.urlopen(url)
    data = uh.read()
    print 'Retrieved',len(data),'characters',data[:20].replace('\n',' ')
    count = count + 1
    try: 
        js = json.loads(str(data))
        # print js  # We print in case unicode causes an error
    except: 
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') : 
        print '==== Failure To Retrieve ===='
        print data
        break

    cur.execute('''INSERT INTO Locations (address, geodata) 
            VALUES ( ?, ? )''', ( buffer(address),buffer(data) ) )
    conn.commit() 
    time.sleep(1)


Found in database  TAMPINES AVE 7 396  Singapore

Found in database  TAMPINES AVE 8 878  Singapore

Found in database  TAMPINES AVE 8 879  Singapore

Found in database  TAMPINES AVE 8 880  Singapore

Found in database  TAMPINES AVE 8 880A  Singapore

Found in database  TAMPINES AVE 8 891  Singapore

Found in database  TAMPINES AVE 8 891A  Singapore

Found in database  TAMPINES AVE 8 892  Singapore

Found in database  TAMPINES AVE 8 892A  Singapore

Found in database  TAMPINES AVE 8 893  Singapore

Found in database  TAMPINES AVE 8 893A  Singapore

Found in database  TAMPINES AVE 9 485A  Singapore

Found in database  TAMPINES AVE 9 485B  Singapore

Found in database  TAMPINES AVE 9 486A  Singapore

Found in database  TAMPINES AVE 9 486B  Singapore

Found in database  TAMPINES AVE 9 486C  Singapore

Found in database  TAMPINES AVE 9 487A  Singapore

Found in database  TAMPINES AVE 9 488A  Singapore

Found in database  TAMPINES AVE 9 491A  Singapore

Found in database  TAMPINES AVE 9 491

In [6]:
def make_string (string):
    result = ''
    for i in string:
        result = result + i.upper()
    return result
def get_lat(unknown_format):
    try:
        lat = unknown_format['results'][0]["geometry"]["location"]["lat"]
        lng = unknown_format['results'][0]["geometry"]["location"]["lng"]
    except:
        lat = np.nan
        lng = np.nan
    return lat,lng
serviceurl = "http://maps.googleapis.com/maps/api/geocode/json?"
conn = sqlite3.connect('geodata_HDB_SG.sqlite')
cur = conn.cursor()
x=pd.read_sql_query('SELECT * FROM Locations',conn)
x['address']=map(make_string,x['address'])
x['geodata']=map(lambda x : json.loads(str(x)),x['geodata'])
x['2']=0
x['1']=0
x[['1','2']]=map(get_lat,x['geodata'])
location_dictt ={}
for i in x['address'].unique():
    location_dictt[i] = [x[x['address']==i].iloc[0]['1'],x[x['address']==i].iloc[0]['2']]


In [7]:
''' some locations here'''
for i in location_dictt.keys()[0:10]:
    print i,', Cordinates:', location_dictt[i]

LOR 1 TOA PAYOH 153A  SINGAPORE , Cordinates: [1.33368, 103.845125]
YISHUN AVE 6 435  SINGAPORE , Cordinates: [1.4214195000000001, 103.8472457]
TELOK BLANGAH HTS 57  SINGAPORE , Cordinates: [1.2746782999999999, 103.81239480000001]
TAMPINES ST 81 819  SINGAPORE , Cordinates: [1.3483894000000001, 103.936696]
TAMPINES ST 41 417  SINGAPORE , Cordinates: [1.3587796999999999, 103.9473507]
FAJAR RD 452  SINGAPORE , Cordinates: [1.385524, 103.77221900000001]
MARINE TER 7  SINGAPORE , Cordinates: [1.3042144, 103.9171011]
HOUGANG AVE 8 445  SINGAPORE , Cordinates: [1.37914, 103.8943029]
CHOA CHU KANG ST 62 606  SINGAPORE , Cordinates: [1.398199, 103.74397]
JURONG WEST ST 64 676B  SINGAPORE , Cordinates: [1.3461029, 103.70380919999999]


In [15]:
'''
define function to calculate distances 
'''
from math import radians, cos, sin, asin, sqrt
def haversine_vector(lat1, lon1, lat2, lon2): 
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    found online, i vectorized it
    """
    # convert decimal degrees to radians 
    lon1 = np.array(map(radians, lon1))
    lat1 = np.array(map(radians, lat1))
    lon2 = np.array(map(radians, lon2))
    lat2 = np.array(map(radians, lat2))
 
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    #print dlat
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(a**.5) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [9]:
'''
Reconnecting to sql database to get GPS data.
'''
conn = sqlite3.connect('geodata_HDB_SG.sqlite')
cur = conn.cursor()
x=pd.read_sql_query('SELECT * FROM Locations',conn)
x['address']=map(make_string,x['address'])
x['geodata']=map(lambda x : json.loads(str(x)),x['geodata']) 
x['2']=0
x['1']=0
x[['1','2']]=map(get_lat,x['geodata'])
 



In [10]:
x.head()

,address,geodata,2,1
0,MARINE DR 60 SINGAPORE,"{u'status': u'OK', u'results': [{u'geometry': ...",103.908327,1.303494
1,JURONG WEST ST 92 925 SINGAPORE,"{u'status': u'OK', u'results': [{u'geometry': ...",103.689240,1.340195
2,JURONG WEST ST 92 926 SINGAPORE,"{u'status': u'OK', u'results': [{u'geometry': ...",103.689132,1.340485
3,JURONG WEST ST 92 928 SINGAPORE,"{u'status': u'OK', u'results': [{u'geometry': ...",103.689385,1.341021
4,JURONG WEST ST 92 929 SINGAPORE,"{u'status': u'OK', u'results': [{u'geometry': ...",103.689168,1.341481


In [11]:
'''
Getting unique location cordinates in a dict
'''
location_dictt ={}
for i in x['address'].unique():
    location_dictt[i] = [x[x['address']==i].iloc[0]['1'],x[x['address']==i].iloc[0]['2']]
    
# use map function to get train data lat and lng
train['lat']=map(lambda x : location_dictt[x][0],train['sql_name'])
train['lng']=map(lambda x : location_dictt[x][1],train['sql_name'])

train.head()

,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,sql_name,lat,lng
0,679,1990-01,GEYLANG,3 ROOM,40,CIRCUIT RD,01 TO 03,57.0,STANDARD,1971,23400.0,CIRCUIT RD 40 SINGAPORE,1.327679,103.886079
1,823,1990-01,JURONG EAST,5 ROOM,209,JURONG EAST ST 21,07 TO 09,123.0,IMPROVED,1983,109000.0,JURONG EAST ST 21 209 SINGAPORE,1.337769,103.740154
2,822,1990-01,JURONG EAST,5 ROOM,258,JURONG EAST ST 24,07 TO 09,121.0,IMPROVED,1983,135000.0,JURONG EAST ST 24 258 SINGAPORE,1.343698,103.739091
3,821,1990-01,JURONG EAST,5 ROOM,240,JURONG EAST ST 21,01 TO 03,121.0,IMPROVED,1982,109000.0,JURONG EAST ST 21 240 SINGAPORE,1.341051,103.741544
4,820,1990-01,JURONG EAST,5 ROOM,240,JURONG EAST ST 21,04 TO 06,121.0,IMPROVED,1982,109000.0,JURONG EAST ST 21 240 SINGAPORE,1.341051,103.741544


In [13]:
mrt=sorted([x for x in location_dictt if 'MRT' in x])
school= [x for x in location_dictt if 'PRIMARY' in x]
mrt_lat= [location_dictt[x][0] for x in mrt if 'MRT' in x]
mrt_lng= [location_dictt[x][1] for x in mrt if 'MRT' in x]

In [16]:
''' 
find nearest mrt to each flat
'''
temp_mrt_data = []
for i in train.index:
    if i%10000==0:
        print i
    ''' 
    following compares ith data point with all mrts stations,and finds the minimm distance
    Vectorized format
    '''
    x=haversine_vector( [train.iloc[i]['lat'],]*len(mrt), [train.iloc[i]['lng'],]*len(mrt),mrt_lat,mrt_lng)
    temp_mrt_data += [[mrt[np.argmin(x)],min(x)],]
train.set_value(train.index,['nearestMRT','dist_nearestMRT'],temp_mrt_data)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000


KeyError: "['nearestMRT' 'dist_nearestMRT'] not in index"

In [17]:
train.to_csv('train_mrt.csv',index=0)